In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
import pandas as pd
from google.cloud import bigquery
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
from google.cloud import storage
from tensorflow.keras import callbacks
from tensorflow.keras import Sequential, Input, layers
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import EarlyStopping

2025-06-04 17:15:27.220038: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-04 17:15:27.221497: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-04 17:15:27.227986: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-04 17:15:27.239807: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749053727.261487  102403 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749053727.26

In [4]:
# Variables
BUCKET_NAME="skin_scan_mohnatz"
GCP_PROJECT = "skin-scan-461716"
BQ_DATASET = "skin_scan"

In [14]:
def get_data_from_bq(
        gcp_project:str,
        query:str,
        data_has_header=True
    ) -> pd.DataFrame:
    """
    Retrieve `query` data from BigQuery
    """
    client = bigquery.Client(project=gcp_project)
    query_job = client.query(query)
    result = query_job.result()
    df = result.to_dataframe()
    return df

query_metadata = f"""
        SELECT *
        FROM `{GCP_PROJECT}`.{BQ_DATASET}.train_metadata
    """

In [15]:
metadata = get_data_from_bq(f"{GCP_PROJECT}", query_metadata)

In [16]:
metadata.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,index
0,HAM_0006706,ISIC_0031725,nv,histo,NaN,female,abdomen,8410
1,HAM_0002610,ISIC_0026876,vasc,consensus,0.0,female,abdomen,2388
2,HAM_0002610,ISIC_0030606,vasc,consensus,0.0,female,abdomen,2389
3,HAM_0003872,ISIC_0028903,nv,consensus,0.0,female,abdomen,9329
4,HAM_0002034,ISIC_0027954,bkl,consensus,5.0,male,abdomen,1035


In [17]:
dx_dict = {"nv": 4,
           "mel":6,
           "bkl":2,
           "bcc":1,
           "akiec":0,
           "vasc":5,
           "df":3   
}

In [18]:
metadata["target"] = metadata.dx.map(dx_dict)

In [19]:
metadata.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,index,target
0,HAM_0006706,ISIC_0031725,nv,histo,NaN,female,abdomen,8410,4
1,HAM_0002610,ISIC_0026876,vasc,consensus,0.0,female,abdomen,2388,5
2,HAM_0002610,ISIC_0030606,vasc,consensus,0.0,female,abdomen,2389,5
3,HAM_0003872,ISIC_0028903,nv,consensus,0.0,female,abdomen,9329,4
4,HAM_0002034,ISIC_0027954,bkl,consensus,5.0,male,abdomen,1035,2


In [21]:
target = metadata[["image_id","target","index"]]
target.head()

,image_id,target,index
0,ISIC_0031725,4,8410
1,ISIC_0026876,5,2388
2,ISIC_0030606,5,2389
3,ISIC_0028903,4,9329
4,ISIC_0027954,2,1035


In [22]:
from google.cloud import storage
from PIL import Image
import io
import matplotlib.pyplot as plt

# Set up the client
client = storage.Client()
bucket = client.bucket(BUCKET_NAME)

blobs = bucket.list_blobs(prefix="train_all_images/")
images = [blob.name for blob in blobs if blob.name.lower().endswith(".jpg")]

In [23]:
import numpy as np
import io
import os  # To handle file paths

def resize(width: int, height: int) -> pd.DataFrame:
    pixel_list = []
    image_ids = []

    for idx, image in enumerate(images):
        print(f"Processing image {idx + 1} of {len(images)}: {image}")

        blob = bucket.blob(image)
        image_bytes = blob.download_as_bytes()
        img = Image.open(io.BytesIO(image_bytes)).convert("RGB")
        img = img.resize((width, height))
        img_array = np.array(img).flatten()
        pixel_list.append(img_array)

        # Extract image ID (e.g., 'ISIC_0024306')
        image_id = os.path.basename(image).split('.')[0]
        image_ids.append(image_id)

    data = np.stack(pixel_list)
    df = pd.DataFrame(data)
    df.insert(0, "image_id", image_ids)  # Insert image_id as first column

    return df

In [24]:
df = resize(96,96)

Processing image 1 of 10015: train_all_images/ISIC_0024306.jpg
Processing image 2 of 10015: train_all_images/ISIC_0024307.jpg
Processing image 3 of 10015: train_all_images/ISIC_0024308.jpg
Processing image 4 of 10015: train_all_images/ISIC_0024309.jpg
Processing image 5 of 10015: train_all_images/ISIC_0024310.jpg
Processing image 6 of 10015: train_all_images/ISIC_0024311.jpg
Processing image 7 of 10015: train_all_images/ISIC_0024312.jpg
Processing image 8 of 10015: train_all_images/ISIC_0024313.jpg
Processing image 9 of 10015: train_all_images/ISIC_0024314.jpg
Processing image 10 of 10015: train_all_images/ISIC_0024315.jpg
Processing image 11 of 10015: train_all_images/ISIC_0024316.jpg
Processing image 12 of 10015: train_all_images/ISIC_0024317.jpg
Processing image 13 of 10015: train_all_images/ISIC_0024318.jpg
Processing image 14 of 10015: train_all_images/ISIC_0024319.jpg
Processing image 15 of 10015: train_all_images/ISIC_0024320.jpg
Processing image 16 of 10015: train_all_images/IS

In [25]:
df.head()

,image_id,0,1,2,3,4,5,6,7,8,...,27638,27639,27640,27641,27642,27643,27644,27645,27646,27647
0,ISIC_0024306,228,131,133,230,130,137,231,124,133,...,137,218,148,135,214,144,133,211,142,128
1,ISIC_0024307,216,121,133,218,122,137,218,125,140,...,145,206,136,141,195,125,126,208,136,141
2,ISIC_0024308,246,173,181,245,171,175,247,171,179,...,162,218,149,158,216,146,155,216,145,155
3,ISIC_0024309,206,118,134,209,120,136,210,120,135,...,120,188,119,122,189,123,126,190,123,129
4,ISIC_0024310,72,33,38,70,32,38,69,31,38,...,45,71,32,43,71,32,45,72,32,43


In [26]:
df = df.merge(target, how="left", on="image_id")

In [28]:
df.head()

,image_id,0,1,2,3,4,5,6,7,8,...,27640,27641,27642,27643,27644,27645,27646,27647,target,index
0,ISIC_0024306,228,131,133,230,130,137,231,124,133,...,148,135,214,144,133,211,142,128,4,4349
1,ISIC_0024307,216,121,133,218,122,137,218,125,140,...,136,141,195,125,126,208,136,141,4,4263
2,ISIC_0024308,246,173,181,245,171,175,247,171,179,...,149,158,216,146,155,216,145,155,4,4217
3,ISIC_0024309,206,118,134,209,120,136,210,120,135,...,119,122,189,123,126,190,123,129,4,3587
4,ISIC_0024310,72,33,38,70,32,38,69,31,38,...,32,43,71,32,45,72,32,43,6,1451


In [30]:
df = df.drop(columns="image_id")

In [32]:
df = df.astype("int64")

In [34]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,27640,27641,27642,27643,27644,27645,27646,27647,target,index
0,228,131,133,230,130,137,231,124,133,229,...,148,135,214,144,133,211,142,128,4,4349
1,216,121,133,218,122,137,218,125,140,217,...,136,141,195,125,126,208,136,141,4,4263
2,246,173,181,245,171,175,247,171,179,247,...,149,158,216,146,155,216,145,155,4,4217
3,206,118,134,209,120,136,210,120,135,208,...,119,122,189,123,126,190,123,129,4,3587
4,72,33,38,70,32,38,69,31,38,67,...,32,43,71,32,45,72,32,43,6,1451


In [37]:
import pandas as pd
from colorama import Fore, Style
from google.cloud import bigquery

In [35]:
GCP_PROJECT = os.environ.get("GCP_PROJECT")
GCP_REGION = os.environ.get("GCP_REGION")

In [36]:
def load_data_to_bq(
        data: pd.DataFrame,
        gcp_project:str,
        bq_dataset:str,
        table: str,
        truncate: bool
    ) -> None:
    """
    - Save the DataFrame to BigQuery
    - Empty the table beforehand if `truncate` is True, append otherwise
    """
    assert isinstance(data, pd.DataFrame)
    full_table_name = f"{gcp_project}.{bq_dataset}.{table}"
    print(Fore.BLUE + f"\nSave data to BigQuery @ {full_table_name}...:" + Style.RESET_ALL)
    client = bigquery.Client(project=gcp_project)
    if truncate == True:
        write_mode = "WRITE_TRUNCATE"
    else:
        write_mode = "WRITE_APPEND"
    job_config = bigquery.LoadJobConfig(write_disposition=write_mode)
    job = client.load_table_from_dataframe(data, full_table_name, job_config=job_config)
    result = job.result()

    print(f"✅ Data saved to bigquery, with shape {data.shape}")

In [ ]:
load_data_to_bq(df,f"{GCP_PROJECT}",f"{BQ_DATASET}","train_96_96",True)


Save data to BigQuery @ skin-scan-461716.skin_scan.train_96_96...:


/home/mohnatz/.pyenv/versions/3.12.9/envs/skin_scan_env/lib/python3.12/site-packages/google/cloud/bigquery/_pandas_helpers.py:489: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(
